<a href="https://colab.research.google.com/github/H-Elizabeth/capstone-dirty-water-notebook/blob/main/V2_Dirty_Water_Project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [125]:
!pip install dash pandas plotly geopandas numpy requests gdown pyheif Pillow piexif exifread --quiet
import dash
from dash import dcc, html, callback_context
from dash.dependencies import Input, Output, State
import pandas as pd
import plotly.graph_objects as go
import geopandas as gpd
import numpy as np
import bisect
import plotly.colors
import plotly.tools as tools
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import patches
import matplotlib.gridspec as gridspec
import requests
import os
import gdown
import shutil
import pyheif
from PIL import Image
import piexif
import exifread
from google.colab import drive, userdata
drive.mount('/content/drive')

# Load the geolabels data
geolabels_link = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vR9KmvTArEvOntjGpzFbpai7tfGCE4atG7cre5BiG_CEhMQw7cOo6bz-SmgJRY7rGCP7ERnRywkwiw7/pub?gid=402113435&single=true&output=csv'
geolabels = pd.read_csv(geolabels_link)

# Load the samples data
samples_link = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vR9KmvTArEvOntjGpzFbpai7tfGCE4atG7cre5BiG_CEhMQw7cOo6bz-SmgJRY7rGCP7ERnRywkwiw7/pub?gid=1821472518&single=true&output=csv'
samples = pd.read_csv(samples_link)

# Load the encampments data
encampments_link = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vR9KmvTArEvOntjGpzFbpai7tfGCE4atG7cre5BiG_CEhMQw7cOo6bz-SmgJRY7rGCP7ERnRywkwiw7/pub?gid=1918593179&single=true&output=csv'
encampments = pd.read_csv(encampments_link)

rain_data_path = '/content/drive/Shareddrives/SSU dirty-water/santa_rosa_rain_data.csv'
rain_gauge_folder = '/content/drive/Shareddrives/SSU dirty-water/Rain Gauge Images/'
site_image_folder = '/content/drive/Shareddrives/SSU dirty-water/Sample Site Images/'

# Download Santa Rosa Creek GeoJSON; Store in session storage
file_id = '1mDhGKaYsRv0Z8pGOVsxYKMmqIvNDhcMr'
output = 'SantaRosaCreek.geojson'
try:
    gdown.download(f'https://drive.google.com/uc?id={file_id}', output, quiet=True)
    srcreek_gdf = gpd.read_file(output)
except:
    srcreek_gdf = gpd.GeoDataFrame()

# Download Rain Data csv; Store in session storage
file_id = '1-4qXImfpTR2R_yRSVCOQAWYbZxRAvGMD'
output = 'santa_rosa_rain_data.csv'
try:
    gdown.download(f'https://drive.google.com/uc?id={file_id}', output, quiet=True)
    cached_rain_data = pd.read_csv(output)
except:
    cached_rain_data = pd.DataFrame()

# Create the session assets folder
assets_folder = '/content/assets'
os.makedirs(assets_folder, exist_ok=True)

# Download the rain gauge images
rain_gauges = [x for x in os.listdir(rain_gauge_folder + '/')]

for file_name in rain_gauges:
    source_file = os.path.join(rain_gauge_folder, file_name)
    destination_file = os.path.join(assets_folder, f"rain_figure_{file_name}")
    # Copy the file
    shutil.copy(source_file, destination_file)

# Download the sample site images
site_images = [x for x in os.listdir(site_image_folder + '/')]

for file_name in site_images:
    source_file = os.path.join(site_image_folder, file_name)
    destination_file = os.path.join(assets_folder, f"site_image_{file_name}.jpeg")
    # Copy the file
    shutil.copy(source_file, destination_file)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [126]:
def dms_to_dd(dms):
    try:  # Accounting for multiple styles of coordinate entries
        dms = dms.replace(" ", "").replace("°", " ").replace("'", " ").replace('"', " ")
        parts = dms.split()
        dd = float(parts[0]) + float(parts[1])/60 + float(parts[2])/(60*60)
        if len(parts) > 3 and parts[3] in ('S','W'):
            dd *= -1
    except:  # The coordinate is already in DD
        dd = float(dms)
    return dd

In [127]:
# Convert Geolabels from DMS to decimal degrees for latitude/longitude
geolabels['Latitude'] = geolabels['Latitude'].apply(dms_to_dd)
geolabels['Longitude'] = geolabels['Longitude'].apply(dms_to_dd)

# Create DateTime column for encampments
encampments['Month'] = encampments['Month'].str.strip()
encampments['Month'] = pd.to_datetime(encampments['Month'], format='%B').dt.month
encampments['date'] = pd.to_datetime(encampments[['Year', 'Month', 'Day']])

# Convert 'HomelessnessScore' to numeric after replacing 'x' with 2
encampments['HomelessnessScore'] = encampments['HomelessnessScore'].replace('x', 2)
encampments['HomelessnessScore'] = pd.to_numeric(encampments['HomelessnessScore'], errors='coerce')

# Add lat/lon coordinates to Encampments
merged_encampments = encampments.merge(geolabels, left_on='EncampmentSite', right_on='Key')

# Remove spaces from 'SampleSite' values
samples['SampleSite'] = samples['SampleSite'].str.replace(' ', '', regex=False)

# Create DateTime column for sample sites
samples['Month'] = samples['Month'].str.strip()
samples['Month'] = pd.to_datetime(samples['Month'], format='%B').dt.month
samples['date'] = pd.to_datetime(samples[['Year', 'Month', 'Day']])

# Add lat/lon coordinates to Samples
merged = samples.merge(geolabels, left_on='SampleSite', right_on='Key')

# Replace no nickname values with sample site code
merged['Nickname'] = merged['Nickname'].fillna(merged['Key'])


sample_site_coordinates = {}

# Add sample sites and their coordinates to the dict
for site in merged['SampleSite'].unique():
    # Extract the Latitude and Longitude values for each water sample site
    lat_lon = merged[merged['SampleSite'] == site][['Latitude', 'Longitude']].iloc[0]
    sample_site_coordinates[site] = (lat_lon['Latitude'], lat_lon['Longitude'])

# Add encampment sites and their coordinates to the dict
for site in merged_encampments['EncampmentSite'].unique():
    # Extract the Latitude and Longitude values for each encampment site
    lat_lon = merged_encampments[merged_encampments['EncampmentSite'] == site][['Latitude', 'Longitude']].iloc[0]
    sample_site_coordinates[site] = (lat_lon['Latitude'], lat_lon['Longitude'])


# List of columns to clean
columns_to_clean = ["pH", "TEMP", "DO(mg/L)", "Conductivity(us/cm)", "Ecoli (MPN/100mL)", "Enterococcus", "D.O%", "Phosphorus"]

# Accounts for 'ND' and '>' in numeric fields
for column in columns_to_clean:
    merged[column] = merged[column].fillna(-1)  # Fill NA values with -1
    merged[column] = merged[column].astype(str)  # Convert the column to string type
    merged[column] = merged[column].replace("ND", "-1")  # Replace "ND" with -1
    merged[column] = merged[column].str.replace(">", "", regex=False)  # Remove ">" character
    merged[column] = merged[column].fillna("-1")
    merged[column] = pd.to_numeric(merged[column], errors='coerce')  # Convert to numeric

# Gets the sorted unique dates for the water sample sites
unique_dates = sorted(merged['date'].unique())

min_date = min(unique_dates)
max_date = max(unique_dates)
date_range = pd.date_range(min_date, max_date)


In [128]:
def send_NOAA_request(start_date, end_date):
    # NOAA API
    url = "https://www.ncei.noaa.gov/cdo-web/api/v2/data"

    # NOAA CDO API Token
    api_token = userdata.get('CDO_token')

    # Define parameters for the GET request
    # Using Data Station: GHCND:USW00023213
    params = {
        "datasetid": "GHCND",               # Daily summaries dataset
        "datatypeid": "PRCP",               # Precipitation data type
        "stationid": "GHCND:USW00094728",
        "startdate": start_date.strftime('%Y-%m-%d'),
        "enddate": end_date.strftime('%Y-%m-%d'),
        "units": "standard",                # Precipitation in inches
        "limit": 1000,                      # Number of results to return (max 1000 per request)
        "format": "json"
    }

    # Headers for the GET request (include the API token)
    headers = {
        "token": api_token
    }

    # Send the GET request
    try:
      response = requests.get(url, headers=headers, params=params, timeout=10)
    except:
      # Prevents request from hanging indefinitely
      print("Request timed out.")
      return None

    # Check if the request was successful
    if response.status_code == 200:
      try:
        rain_data = response.json()
        if 'results' in rain_data:
          results = rain_data['results']
          return pd.DataFrame(results)
        else:
          print("No results found in the response.")
          return None
      except ValueError:
        print("Failed to parse JSON response.")
        return None
    else:
      # If the request fails, print the status code and error message
      print(f"Failed to retrieve data. Status code: {response.status_code}")
      print(f"Error message: {response.text}")
      return None

def update_rain_data():
    if 'date' in cached_rain_data.columns:
        cached_rain_data['date'] = pd.to_datetime(cached_rain_data['date'], format='ISO8601')
        last_saved_date = cached_rain_data['date'].max()
    else:
        last_saved_date = min_date - pd.Timedelta(days=1)

    start_date = last_saved_date
    end_date = start_date
    updated_rain_data = cached_rain_data     # Preparing for dataset concatenation
    today = pd.Timestamp.today().normalize()
    if end_date == today:
        return updated_rain_data

    while end_date != today:
      start_date = start_date + pd.Timedelta(days=1)
      end_date = start_date + pd.DateOffset(years=1)
      if end_date > today:
          end_date = today
      print(f"Fetching data for [{start_date}, {end_date}]")
      new_data = send_NOAA_request(start_date, end_date)
      if new_data is None:
          print("Failed to retrieve data.")
          break
      else:
          updated_rain_data = pd.concat([updated_rain_data, new_data], axis=0)
      start_date = end_date

    updated_rain_data[['date', 'datatype', 'station', 'value']].to_csv(rain_data_path, index = False)
    return updated_rain_data

In [129]:
def generate_rain_figures():
    for sample_date in unique_dates:
      # Get the rain data from the past week before sample date
      one_week_prior = sample_date - pd.Timedelta(days = 7)
      df = cached_rain_data[(cached_rain_data['date'] >=  one_week_prior) & (cached_rain_data['date'] <= sample_date)]
      rain_figures[sample_date] = df

    for sample_date, rain_df in rain_figures.items():
      # Update the rain_figures dictionary with the path to the figure
      rain_figures[sample_date] = f'/assets/rain_figure_{sample_date.strftime("%Y-%m-%d")}.png'

      if rain_df.shape[0] == 0:
          blank_image = Image.new('RGB', (400, 300), 'white')
          blank_image.save(rain_figures[sample_date])
          return

      sample_date_label = sample_date.strftime('%Y-%m-%d')

      if f'{sample_date_label}.png' not in rain_gauges:
          rain_df = rain_df.copy()
          rain_df['formatted_date'] = rain_df['date'].dt.strftime('%m-%d')  # Month-Day format

          # Calculate total inches of rain over the 7-day period
          total_inches = round(rain_df['value'].sum(), ndigits=2)

          # Customize width ratios of subplots
          gs = gridspec.GridSpec(1, 2, width_ratios=[1, 2])  # First subplot smaller, second larger
          fig = plt.figure(figsize=(8, 6))  # Set overall figure size

          # Create the Rain Gauge Visualization in the first subplot
          ax0 = fig.add_subplot(gs[0])
          gauge_height = 8  # Total height of the gauge
          rain_level = min((total_inches / 10) * gauge_height, gauge_height)  # Calculate fill height

          # Draw the gauge outline
          gauge_outline = patches.FancyBboxPatch((0.25, 0), 0.5, gauge_height, boxstyle="round,pad=0.05", linewidth=2, edgecolor='black', facecolor='none')
          ax0.add_patch(gauge_outline)

          # Draw the rain level with a gradient-like fill
          rain_color = 'deepskyblue' if total_inches <= 10 else 'orange'  # Change color if rainfall exceeds 10 inches
          ax0.add_patch(patches.Rectangle((0.25, 0), 0.5, rain_level, fill=True, color=rain_color, alpha=0.7))
          ax0.add_patch(patches.Rectangle((0.25, rain_level), 0.5, 0.15, fill=True, color='lightblue', alpha=0.5))

          # Add ticks for rainfall thresholds
          thresholds = [1, 3, 5, 7, 10]  # Set thresholds
          for threshold in thresholds:
              y_position = (threshold / 10) * gauge_height
              ax0.axhline(y=y_position, xmin=0.15, xmax=0.85, color='red', linestyle='--', linewidth=1)  # Draw the threshold line
              ax0.text(0.18, y_position + .1, f'{threshold} in', horizontalalignment='right', fontsize=20, color='red')

          # Set limits and remove axes for a cleaner look
          ax0.set_xlim(0, 1)
          ax0.set_ylim(0, gauge_height + 2)
          ax0.axis('off')  # Turn off the axis
          ax0.set_title(f'Total Rain in Past Week:\n{total_inches} Inches\n',
                          fontsize=20, fontweight='bold', color='darkblue', y=.95)

          # Bar Graph for Daily Rainfall in the second subplot
          ax1 = fig.add_subplot(gs[1])  # Second subplot (larger)
          sns.barplot(data=rain_df, x='formatted_date', y='value', color='blue', alpha=0.7, ax=ax1)
          ax1.set_title(f'Daily Rain Totals', color='darkblue', fontsize=20, fontweight='bold')
          ax1.set_xlabel('', fontsize=1)
          ax1.set_ylabel('Inches', fontsize=24)
          ax1.tick_params(axis='x', rotation=45)
          ax1.tick_params(axis='both', labelsize=20)
          ax1.grid()

          plt.tight_layout()  # Adjust layout

          # Define the file path for the PNG file
          file_path = f'/content/assets/rain_figure_{sample_date.strftime("%Y-%m-%d")}.png'
          fig.savefig(file_path, bbox_inches='tight')
          plt.close(fig)

In [130]:
def euclidean_distance(lat1, lon1, lat2, lon2):
    return ((lat1 - lat2)**2 + (lon1 - lon2)**2)**0.5

# Function to find the closest site from the dictionary
def find_closest_site(lat, lon, sample_sites_dict):
    closest_site = None
    min_distance = float('inf')

    for site, (site_lat, site_lon) in sample_sites_dict.items():
        # Calculate the Euclidean distance between the input coordinates and the current site
        distance = euclidean_distance(lat, lon, site_lat, site_lon)

        # Update the closest site if this distance is smaller
        if distance < min_distance:
            closest_site = site
            min_distance = distance

    return closest_site

In [131]:
cached_rain_data = update_rain_data()
cached_rain_data['date'] = pd.to_datetime(cached_rain_data['date'], format='ISO8601')
rain_figures = {}

generate_rain_figures()

Fetching data for [2024-10-31 00:00:00, 2024-11-03 00:00:00]
No results found in the response.
Failed to retrieve data.


In [132]:
def extract_metadata(heic_path):
    heif_file = pyheif.read(heic_path)

    # Extract EXIF metadata from HEIC
    exif_data = heif_file.metadata or []
    exif_dict = {}
    for metadata in exif_data:
        if metadata['type'] == 'Exif':
            exif_dict = piexif.load(metadata['data'])

    # Convert HEIC to JPEG
    image = Image.frombytes(
        heif_file.mode,
        heif_file.size,
        heif_file.data,
        "raw",
        heif_file.mode,
        heif_file.stride,
    )

    # Check for the orientation tag in the EXIF metadata
    orientation = exif_dict.get('0th', {}).get(piexif.ImageIFD.Orientation, 1)

    # Rotate the image based on orientation value
    if orientation == 3:
        image = image.rotate(180, expand=True)
    elif orientation == 6:
        image = image.rotate(270, expand=True)
    elif orientation == 8:
        image = image.rotate(90, expand=True)

    # Convert the exif_dict back to bytes
    exif_bytes = piexif.dump(exif_dict)

    # Save it as a temporary JPEG image to extract EXIF metadata
    temp_jpeg = "temp_image.jpg"
    image.save(temp_jpeg, "JPEG", exif=exif_bytes)

    # Open the saved JPEG and extract metadata
    with open(temp_jpeg, 'rb') as img_file:
        tags = exifread.process_file(img_file)

    # Extract GPS and DateTime information if available
    gps_lat = tags.get('GPS GPSLatitude')
    gps_lon = tags.get('GPS GPSLongitude')
    datetime = tags.get('EXIF DateTimeOriginal')

    # Convert GPS coordinates to decimal format
    def convert_to_degrees(value):
        d = float(value.values[0].num) / float(value.values[0].den)
        m = float(value.values[1].num) / float(value.values[1].den)
        s = float(value.values[2].num) / float(value.values[2].den)
        return d + (m / 60.0) + (s / 3600.0)

    if gps_lat and gps_lon:
        lat = convert_to_degrees(gps_lat)
        lon = convert_to_degrees(gps_lon) * -1
    else:
        lat, lon = None, None

    metadata = {"latitude": lat, "longitude": lon, "datetime": datetime}

    sample_site = find_closest_site(metadata['latitude'], metadata['longitude'], sample_site_coordinates)
    timestamp = metadata['datetime'].values.replace(':', '-', 2)
    timestamp = pd.to_datetime(timestamp).strftime('%Y-%m-%d')

    image.save(f'/content/drive/Shareddrives/SSU dirty-water/Sample Site Images/{sample_site}_{timestamp}', "jpeg", exif=exif_bytes)
    os.remove(heic_path)

# Collect all the .heic files in the SSU dirty-water folder
heic_files = [x for x in os.listdir('/content/drive/Shareddrives/SSU dirty-water/') if x.lower().endswith('.heic')]

for file in heic_files:
  extract_metadata('/content/drive/Shareddrives/SSU dirty-water/' + file)
if os.path.isfile('/content/temp_image.jpg'):
  os.remove('/content/temp_image.jpg')

In [134]:
# Create a Dash app
app = dash.Dash(__name__)

color_dict = {
    0: 'rgba(255, 255, 255, .5)',  # No homeless
    1: 'rgba(27, 77, 62, .1)',  # Homeless
    2: 'rgba(0, 0, 0, 0)'  # Not monitored
}

description_dict = {0: 'Monitored but no homelessness', 1: 'Monitored and homelessness found', 2: 'Not monitored'}

# Color Key
color_ranges = {}

initial_rain_gauge = rain_figures.get(pd.Timestamp(unique_dates[0]))

# Check if the image exists
if not os.path.exists(initial_rain_gauge):
    initial_rain_gauge = None

app.layout = html.Div([
html.Div(
  style={
      'backgroundColor': '#f0f0f0',
      'borderRadius': '10px',
      'padding': '10px',
      'width': '98%',
      'boxShadow': '0 4px 8px rgba(0, 0, 0, 0.2)',
      'display': 'flex',  # Use flexbox for alignment
      'alignItems': 'center',  # Vertically align items in the center
      'justifyContent': 'flex-start'  # Align items to the start
  },
  children=[
      html.Label('Select Date:', style={'fontSize': 24, 'textAlign': 'left', 'marginRight': '20px'}),
      html.Div(
          style={
              'flex': '1',  # Allow the div to flex
              'minWidth': '300px',  # Minimum width for the slider container
              'marginLeft': '20px',  # Space between the label and slider
              'marginRight': '20px'
          },
          children=[
              dcc.Slider(
                  id='date-slider',
                  min=0,
                  max=len(unique_dates) - 1,
                  value=0,
                  marks={i: {'label': f"{pd.Timestamp(date).strftime('%b, %Y')}", 'style': {'whiteSpace': 'nowrap', 'color': 'Black'}} for i, date in enumerate(unique_dates)},
                  step=None,
                  updatemode='drag',
                  included=False,
                  vertical=False
              )
          ]
      )
  ]
),
    html.Button('Start/Stop', id='start-button', n_clicks=0, style={'marginBottom': '10px'}),
    html.Button('Switch Map Style', id='switch-button', n_clicks=0),
    dcc.Checklist(
        id='encampment-toggle',
        options=[{'label': 'Show Encampments', 'value': 'SHOW'}],
        value=['SHOW']
    ),
    dcc.Interval(
        id='interval-component',
        interval=3 * 1000,  # in milliseconds
        max_intervals=0  # Max intervals set to 0 will stop the interval
    ),
    html.Div([
        # Map and Dropdown + Color Key container
        html.Div([
            dcc.Graph(id='map', style={'width': '100%', 'height': '500px'}),
            dcc.Markdown(id='debug-output', style={'whiteSpace': 'pre-line'}),
            dcc.Store(id='zoom-level', data=12),
            dcc.Store(id='lat-lon', data={'lat': 38.45, 'lon': -122.7}),
        ], style={'width': '75%', 'display': 'inline-block'}),

        html.Div([
                html.Img(id='rain-gauge', src=initial_rain_gauge, style={'width': '96%', 'display': 'block', 'paddingTop': '0px',
                                                                         'paddingRight': '10px', 'paddingBottom': '10px', 'paddingLeft': '10px'}),
                dcc.Dropdown(
                id='color-dropdown',
                options=[
                    {'label': 'pH', 'value': 'pH'},
                    {'label': 'TEMP', 'value': 'TEMP'},
                    {'label': 'DO(mg/L)', 'value': 'DO(mg/L)'},
                    {'label': 'Conductivity(us/cm)', 'value': 'Conductivity(us/cm)'},
                    {'label': 'Phosphorus', 'value': 'Phosphorus'},
                    {'label': 'Ecoli (MPN/100mL)', 'value': 'Ecoli (MPN/100mL)'},
                    {'label': 'Enterococcus', 'value': 'Enterococcus'}
                ],
                value='Ecoli (MPN/100mL)',  # Default value
                style={'width': '98%', 'margin-left': '5px'}
            ),
            html.Div(id='color-key', style={'border': 'thin lightgrey solid', 'marginLeft': '10px', 'marginRight': '2px',
                                            'padding': '10px', 'marginTop': '5px'})
        ], style={'width': '25%', 'display': 'inline-block', 'vertical-align': 'top'})
    ], style={'display': 'flex', 'flex-direction': 'row'}),
    html.Img(id='image', src=None, style={'width': '95%', 'padding': '20px', 'textAlign': 'center'}),
])


# Initialize a variable to keep track of the last clicked point
last_clicked_point = None

@app.callback(
    Output('color-key', 'children'),
    Input('color-dropdown', 'value')
)
def update_color_key(selected_param):
    color_descriptions = color_ranges.get(selected_param, [])
    spans = []
    for color, description in color_descriptions:
        spans.extend([
            html.Span(style={'display': 'inline-block', 'width': '20px', 'height': '20px', 'marginRight': '5px', 'backgroundColor': color}),
            html.Span(description),
            html.Br()
        ])
    return spans

@app.callback(
    Output('interval-component', 'max_intervals'),
    Input('start-button', 'n_clicks')
)
def start_cycle(n_clicks):
    if n_clicks % 2 == 0:
        return 0  # stop cycling
    else:
        return -1  # start cycling

# Define callback for the interval component
@app.callback(
    Output('date-slider', 'value'),
    Input('interval-component', 'n_intervals'),
    State('date-slider', 'value')
)
def update_slider(n_intervals, current_value):
    if n_intervals is None:
        # When the app starts, n_intervals is None, so we need to check for this
        return current_value
    elif n_intervals > 0:
        return (current_value + 1) % len(unique_dates)  # cycle through dates
    else:
        return current_value  # keep current value

# Define the callback to update the rain gauge image
@app.callback(
    Output('rain-gauge', 'src'),
    Input('date-slider', 'value')
)
def update_rain_gauge(selected_date_index):
    sample_date = unique_dates[selected_date_index]
    image_path = rain_figures.get(pd.Timestamp(sample_date))
    if image_path:
        return image_path
    return None

# This function takes the color brewer palette name and type (sequential or diverging) and
# returns a list of evenly spaced colors from that palette
def get_even_colors(palette, palette_type, num_colors):
    num_colors -= 1 # This accounts for "No Data" color, which is a transparent black
    # Sample num_colors evenly spaced colors from the given palette.
    if palette_type == 'sequential':
        palette = getattr(plotly.colors.sequential, palette)
        palette = palette[2:] # Exclude the first 2 colors in palette, since they are often too light to show up on background
    elif palette_type == 'diverging':
        palette = getattr(plotly.colors.diverging, palette)
    else:
        raise ValueError(f"Palette type '{palette_type}' is not recognized")
    # Calculate evenly spaced indices
    indices = [int(i * (len(palette) - 1) / (num_colors - 1)) for i in range(num_colors)]
    return ['rgba(0, 0, 0, 0.62)'] + [palette[i] for i in indices]

# Setup color mapping for sample points
color_mapping = {
    'pH': {
        'ranges': [0, 6.4, 8.5],
        'ranges_descr': ['No Data', '0-6.4 Acidic', '6.5-8.5 Normal', '8.6-14 Basic'],
        'colors': 'RdYlBu',
        'palette': 'diverging'
    },
    'DO(mg/L)': {
        'ranges': [0, 5, 6],
        'ranges_descr': ['No Data', '0-5 Very low', '5-6 Low', '>6 Ideal'],
        'colors': 'YlOrRd_r',
        'palette': 'sequential'
    },
    'Conductivity(us/cm)': {
        'ranges': [0, 200, 400, 600, 800],
        'ranges_descr': ['No Data', '0-200', '200-400', '400-600', '600-800', '>800'],
        'colors': 'Blues',
        'palette': 'sequential'
    },
    'Phosphorus': {
        'ranges': [0, 0.04, 0.06, 0.1, 0.15],
        'ranges_descr': ['No Data', '<0.04 Very low', '0.04-0.06 Low', '0.06-0.1 Moderate', '0.1-0.15 High', '>0.15 Very high'],
        'colors': 'YlOrRd',
        'palette': 'sequential'
    },
    'Ecoli (MPN/100mL)': {
        'ranges': [0, 100, 300, 1000, 2419],
        'ranges_descr': ['No Data', '0-100 Low', '100-300 Elevated', '300-1000 High', '1000 - 2419 Very High', '>2419 Too High to Measure'],
        'colors': 'Reds',
        'palette': 'sequential'
    },
    'D.O%': {
        'ranges': [0, 80, 100],
        'ranges_descr': ['No Data', '0-80 Very low', '80-100 Low', '>100 Ideal'],
        'colors': 'RdYlBu',
        'palette': 'diverging'
    },
    'TEMP': {
        'ranges': [0, 11, 13, 15],
        'ranges_descr': ['No Data', '0-11', '11-13', '13-15', '>15'],
        'colors': 'PuBu',
        'palette': 'sequential'
    },
    'Enterococcus': {
        'ranges': [0, 300, 400, 500],
        'ranges_descr': ['No Data', '0-300', '300-400', '400-500', 'Enterococcus > 500'],
        'colors': 'BuPu',
        'palette': 'sequential'
    }
}

# Update the dictionary with list of colors from the selected color brewer palette
for param, mapping in color_mapping.items():
    num_colors = len(mapping['ranges']) + 1
    mapping['colors'] = get_even_colors(mapping['colors'], mapping['palette'], num_colors)
    # print(param, ':', mapping['colors'])

def map_colors(param, value):
    if param not in color_mapping:
        return 'black'  # Default color if param is not found

    # Get the ranges and colors for the given param
    ranges = color_mapping[param]['ranges']
    colors = color_mapping[param]['colors']

    # Use bisect to find the index of the range
    index = bisect.bisect_left(ranges, value)

    # Return the corresponding color
    return colors[index]

# Update the Color Ranges Key with colors
for param, mapping in color_mapping.items():
    ranges_descr = mapping['ranges_descr']
    colors = mapping['colors']

    # Initialize an empty list to store the tuples
    color_ranges[param] = []

    # Iterate through the indices of the colors and descriptions
    for i in range(len(colors)):
        color = colors[i]
        description = ranges_descr[i]
        # Append the tuple to the list
        color_ranges[param].append((color, description))

@app.callback(
    [Output('image', 'src'),
     Output('debug-output', 'children')],
    [Input('map', 'clickData')]
)
def show_site_image_on_click(click):
    if click:
        point = click['points'][0]
        if 'customdata' in point and point['customdata']:
            site_name = point['customdata'][0]
            sample_date = point['customdata'][1].split('T')[0]
            file_name = f"site_image_{site_name}_{sample_date}.jpeg"
            if file_name in os.listdir('/content/assets'):
                return f"/assets/{file_name}", file_name
    return None, ''

# Callback to update the map when the slider value changes
@app.callback(
    [Output('map', 'figure'),
    Output('lat-lon', 'data'),
    Output('zoom-level', 'data')],
    [Input('date-slider', 'value'),
     Input('encampment-toggle', 'value'),
     Input('color-dropdown', 'value'),
     Input('switch-button', 'n_clicks'),
     Input('map', 'relayoutData')],
     [State('lat-lon', 'data'),
     State('zoom-level', 'data')]
)
def update_map(selected_date_index, encampment_toggle_value, color_value, n_clicks, relayout_data, lat_lon, current_zoom):
    fig = go.Figure()

    # Check which input triggered the callback
    ctx = callback_context

    if ctx.triggered:
        triggered_input = ctx.triggered[0]['prop_id'].split('.')[0]
        # Log the triggered input to a file
        message = f"Callback triggered by: {triggered_input}"


    # Get the current latitude and longitude from the stored data
    current_lat = lat_lon['lat']
    current_lon = lat_lon['lon']
    zoom_level = current_zoom

    # Use the new center and zoom level from relayoutData if it exists
    if relayout_data:
        new_zoom = relayout_data.get('mapbox.zoom', current_zoom)

        if 'mapbox.center' in relayout_data and triggered_input == 'map':
            new_lat = relayout_data['mapbox.center']['lat']
            new_lon = relayout_data['mapbox.center']['lon']
            return dash.no_update, {'lat': new_lat, 'lon': new_lon}, zoom_level

        # Zoom change detected: proceed with map update
        if new_zoom != current_zoom:
            zoom_level = new_zoom

    # Calculate the shift value based on the zoom level
    # shift_value = calculate_latitude_shift(zoom_level)
    shift_value = 0.00182

    selected_date = unique_dates[selected_date_index]  # Get the selected date using the index

    included_samples = merged[merged['date'] <= selected_date]

    # Preprocess the DataFrame to create a custom hover text column
    no_data_indicator = 'No Data'
    included_samples['hover_text'] = included_samples.apply(lambda row: (
        f"<b>Sample Site: {row['SampleSite']}</b><br>"
        f"Date: {row['date'].strftime('%Y-%b-%d')}</b><br>"
        f"pH: {row['pH'] if row['pH'] != -1 else no_data_indicator}<br>"
        f"TEMP: {row['TEMP'] if row['TEMP'] != -1 else no_data_indicator}<br>"
        f"DO(mg/L): {row['DO(mg/L)'] if row['DO(mg/L)'] != -1 else no_data_indicator}<br>"
        f"Conductivity(us/cm): {row['Conductivity(us/cm)'] if row['Conductivity(us/cm)'] != -1 else no_data_indicator}<br>"
        f"Phosphorus: {row['Phosphorus'] if row['Phosphorus'] != -1 else no_data_indicator}<br>"
        f"Ecoli (MPN/100mL): {row['Ecoli (MPN/100mL)'] if row['Ecoli (MPN/100mL)'] != -1 else no_data_indicator}<br>"
        f"Enterococcus: {row['Enterococcus'] if row['Enterococcus'] != -1 else no_data_indicator}<br>"),
    axis=1)

    included_encampments = merged_encampments[merged_encampments['date'] <= selected_date]

    if not included_samples.empty:
        last_sample_date = included_samples['date'].max()
        included_samples = included_samples[included_samples['date'] == last_sample_date]

    if not included_encampments.empty:
        last_encampment_date = included_encampments['date'].max()
        included_encampments = included_encampments[included_encampments['date'] == last_encampment_date]

    descriptions = encampments['HomelessnessScore'].map(description_dict)
    colors = included_encampments['HomelessnessScore'].map(color_dict)
    included_encampments['color'] = colors

    if color_value in included_samples.columns:
        included_samples['color'] = included_samples[color_value].apply(lambda value: map_colors(color_value, value))
    else:
        included_samples['color'] = 'blue'  # default color if color_value is not a valid column

    if color_value in included_encampments.columns and color_value != 'HomelessnessScore':
        included_encampments['color'] = included_encampments[color_value].apply(lambda value: map_colors(color_value, value))
    elif 'HomelessnessScore' in included_encampments.columns:
        included_encampments['color'] = included_encampments['HomelessnessScore'].map(color_dict)
    else:
        included_encampments['color'] = 'red'  # default color if color_value is not a valid column

    # Draw Santa Rosa Creek on map
    # Initialize list for LineString traces
    lines = []

    # Extract coordinates for LineStrings
    for geom in srcreek_gdf.geometry:
        if geom.geom_type == 'LineString' and not geom.is_empty:
            x, y = geom.xy
            # Ensure x and y are lists of longitudes and latitudes
            lon = list(x)
            lat = list(y)

            # Append the LineString trace to the lines list
            lines.append(go.Scattermapbox(
                lon=lon,
                lat=lat,
                mode='lines',
                line=dict(width=2, color='rgba(102, 179, 255, 0.6)'),
                hoverinfo='none',
                name='Storm Drain Lines',
                showlegend=False
            ))
            # # Add LineStrings to the figure
    for line in lines:
        fig.add_trace(line)

    # Draw points for encampments
    if 'SHOW' in encampment_toggle_value:
        included_encampments = included_encampments[included_encampments['HomelessnessScore'] == 1]
        fig.add_trace(go.Scattermapbox(
            lat=included_encampments['Latitude'],
            lon=included_encampments['Longitude'],
            mode='markers',
            marker=dict(symbol="campsite", color=included_encampments['color'], size=10),
            text=included_encampments.apply(lambda
                                                row: f"Encampment Site: {row['EncampmentSite']}<br>{descriptions[row.name]}<br>Notes: {row['Notes'] if pd.notna(row['Notes']) else 'None'}",
                                            axis=1),
            hoverinfo='text',
            # custom_data=included_encampments[['EncampmentSite','date']],
            name="Encampments",
            showlegend=False
        ))

    # Draw lines from original points to shifted points (Water Sample Data)
    for index, row in included_samples.iterrows():
        orig_lat = row['Latitude']
        orig_lon = row['Longitude']
        shifted_lat = orig_lat + shift_value
        shifted_lon = orig_lon

        fig.add_trace(go.Scattermapbox(
            lat=[orig_lat, shifted_lat],
            lon=[orig_lon, shifted_lon],
            mode='lines',
            line=dict(width=2, color='rgba(0, 0, 0, .5)'),
            hoverinfo='skip',
            text='',
            name='',
            showlegend=False
        ))

    # Draw points for Water Sample Data
    fig.add_trace(go.Scattermapbox(
      lat=included_samples['Latitude'] + shift_value,
      lon=included_samples['Longitude'],
      mode='markers',
      marker=dict(
          symbol="circle",
          color=included_samples['color'],
          size=12,
          opacity=.8,
          sizemode='area'
      ),
      customdata=included_samples[['SampleSite', 'date', 'pH', 'TEMP', 'DO(mg/L)', 'Conductivity(us/cm)', 'Phosphorus', 'Ecoli (MPN/100mL)', 'Enterococcus']],
      text=included_samples['hover_text'],
      name="",
      showlegend=False
    ))

    fig.update_layout(
        mapbox_style="light",
        mapbox_layers=[],
        mapbox=dict(center=dict(lat=current_lat, lon=current_lon),
                    zoom=zoom_level,
                    accesstoken=userdata.get('Mapbox_token')
                    ),
        margin={"r": 0, "t": 0, "l": 0, "b": 0}
    )

    return fig, lat_lon, zoom_level


app.run_server(mode='inline', debug=True)

<IPython.core.display.Javascript object>